In [4]:
import nest_asyncio
import asyncio



In [6]:
nest_asyncio.apply()

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent, ToolCallResult, ToolCall
from llama_index.core.workflow import Context
from llama_index.llms.openai import OpenAI  # Replace if using another LLM



# Set your LLMs (you can switch between Ollama or OpenAI etc.)
llm = Ollama(model="llama3:instruct", request_timeout=120.0)
Settings.llm = llm




In [7]:
# Create MCP client and get available tools
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool_spec = McpToolSpec(client=mcp_client)

In [8]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with our Book Library Database.
"""

In [9]:
async def get_agent(tool_spec: McpToolSpec) -> FunctionAgent:
    tools = await tool_spec.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="Agent that can interact with the Book Library MCP tool server.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent


In [11]:
async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = True,
):
    handler = agent.run(message_content, ctx=agent_context)

    async for event in handler.stream_events():
        if verbose and isinstance(event, ToolCall):
            print(f"🔧 Calling tool: {event.tool_name} | Params: {event.tool_kwargs}")
        elif verbose and isinstance(event, ToolCallResult):
            print(f"✅ Tool {event.tool_name} returned: {event.tool_output}")

    response = await handler
    return str(response)

async def main():
    agent = await get_agent(mcp_tool_spec)
    context = Context(agent)

    print("🤖 Agent is ready. Type your queries or type 'exit' to quit.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            break
        response = await handle_user_message(user_input, agent, context, verbose=True)
        print("Agent:", response)

if __name__ == "__main__":
    asyncio.run(main())

🤖 Agent is ready. Type your queries or type 'exit' to quit.
